Leer Texto de imagenes

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import csv
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Obtener la lista de archivos en la carpeta
folder_path = 'data/input/imagenes_prueba/'
image_files = os.listdir(folder_path)

# Cargar el procesador y el modelo
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten').to(device)

# Iterar sobre cada imagen en la carpeta
for image_file in image_files:
    if image_file.endswith('.png') or image_file.endswith('.jpg') or image_file.endswith('.jpeg'):
        # Cargar la imagen
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)

        # Preprocesamiento de la imagen
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        
        # Añadir una dimensión adicional para que la imagen tenga 3 dimensiones
        binary_image = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)

        # Convertir la imagen a tensor y enviarla a la GPU si está disponible
        image_tensor = torch.tensor(binary_image, device=device)

        # Procesar la imagen y generar el texto
        pixel_values = processor(images=image_tensor, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Imprimir el texto generado
        print("Texto reconocido en", image_file, ":", generated_text)

        # Escribir el texto reconocido en un archivo CSV
        output_csv = 'data/output/output_text/output_text_trOCR.csv'
        with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:  # Use 'a' for append mode
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow([image_file, generated_text])

print("Texto reconocido de todas las imágenes guardado en", output_csv)